In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def LIF(I):
    """
    Runs a LIF simulation on neuron and returns outputted voltage

            Parameters:
                    I (double[]): A list of input voltages in mV
                    Cm (double): The membrane capacitance
                    Rm (double): The membrane resistance

            Returns:
                    V (double[]): A list of the output voltages in mV
    """
    V_thresh = 30
    V_rest = -65
    V_spike = 80
    dT = 0.02  # time step in ms
    total_time = (I.size) * dT
    Cm=4
    Rm=5
    
    # an array of time
    time = np.arange(0, total_time, dT)

    # default voltage list set to resting volatage of -65mV
    V = (-65) * np.ones(len(time))
    #V_bin = 0*np.ones(len(time))

    did_spike = False

    # function member variable to track spikes
    spikes = 0

    for t in range(len(time)):
        # using "I - V(t)/Rm = Cm * dV/dT"
        dV = (I[t] - (V[t - 1] - V_rest) / Rm) / Cm

        # reset membrane potential if neuron spiked last tick
        if did_spike:
            V[t] = V_rest + dV * dT
        else:
            V[t] = V[t - 1] + dV * dT

        # check if membrane voltage exceeded threshold (spike)
        if V[t] > V_thresh:
            did_spike = True
            #V_bin[t] = 80
            # set the last step to spike value
            V[t] = V_spike
            spikes += 1
        else:
            did_spike = False
    spike_rate=spikes/100*100
    return V,spikes

In [5]:
def spktrain2spks(V):
    js=np.zeros(V.size)
    for i in range (V.size):
        if V[i]==80:
            js[i]=80
    return js

In [166]:
def generateSpikeRates(input):
    
    """
    Encodes input into resulting spike rates for all timesteps in a time frame given input current injected 
    Parameters:
        input: Array of inputs at timesteps t
    Returns:
        spike_rate: Array of spiking rates at timesteps t
    """
    t = 100
    dT = .01
    time = np.arange(0, t + dT, dT)
    if isinstance(input, np.integer):
        spike_rate = np.zeros(1)
        I = input*np.ones(len(time))
        V,spikes=LIF(I=I)
        spike_rate[0] = spikes / (time[len(time)-1] - time[0] ) * 100
        return spike_rate
    spike_rate = np.empty(input.size).astype(float)
    for i in range(input.size):
        """calculate spike rate for each iteration"""

        I = input[i]*np.ones(len(time))

        # run LIF simulation
        V,spikes=LIF(I=I)

        # calculate the spike rate during the period
        spike_rate[i] = spikes / (time[len(time)-1] - time[0] ) * 100
        
    return spike_rate

def encodeInputs(inputarray):
    """
    Encodes neuron activity from x and y into rates generated from input current injected at each dT
    Parameters:
        x: Array of binary values representing x
        y: Array of binary values representing y
    Returns:
        x_out: Inputs of x encoded by their corresponding firing rates
        y_out: Inputs of y encoded by their corresponding firing rates
    """
    # example: [1 0 1 0] becomes [107 58 107 58]
    one_curr = 107
    zero_curr = 58

    #encoding for logic
    #inputToCurr = lambda t: zero_curr if t == 0 else (one_curr if t == 1 else -1)
    
    #encoding for black/white
    inputToCurr = lambda t: zero_curr if t <= 7 else (one_curr if t >7  else -1)

    
    out=inputarray
    out=np.multiply(out,one_curr-zero_curr)
    out=np.add(out,zero_curr)
    
    #equation for numbers
    #x_in = np.array([inputToCurr(xi) for xi in inputarray])

    x_out=np.zeros(inputarray.shape)
    #x_out = generateSpikeRates(x_in)
    for i in range(len(x_out)):
        x_out[i]=generateSpikeRates(out[i])
#    y_out = generateSpikeRates(y_in)

    return x_out

In [13]:
x_out=encodeInputs([[0,0], [0,1], [1,0],[1,1]])
print(len(x_out))
# np.zeros([2,4])

4


In [123]:
def train(inputs,target,epoch,l_rt):
    print("---------------training---------------")
    eout = encodeInputs(inputs)
    teach = encodeInputs(target)
    #inputs=[[0,0], [0,1], [1,0],[1,1]]
    #weights[0] = w_x,out; weights[1] = w_y,out
    weights = 0*np.ones([len(inputs[0])]).astype(float)

    for epoch in range(epoch):

#         o_input = np.dot(inputs, weights) # [w_sum1, w_sum2, w_sum3, w_sum4]
        print("epoch: ", epoch)

#         o_out = generateSpikeRates(o_input) #generates spike rates from corresponding input currents calculated for o_input
#         print("output firing rates: \n", o_out)

        for i in range(len(inputs)): 
            for j in range(len(weights)):
                weights[j] += l_rt*eout[i][j]*teach[i] #standard Hebb model modified for perceptrons/teacher neuron => Δw=a*v_pre*(target-v_post)
            print(weights)
        
    print("--------------------------------------")

    return weights

In [21]:
def predict(input, weights, thresh, one_rt=51, zero_rt=25):
    
    print("input: ", input)
    
    p_sum = np.dot(input, weights)
    p_rt = generateSpikeRates(np.array([p_sum]))
    
    #zero_lo = zero_rt - thresh
    #zero_hi = zero_rt + thresh
    
    one_lo = one_rt - thresh
    one_hi = one_rt + thresh
    prediction = 0
    if one_lo <= p_rt <= one_hi:
        prediction = 1

    print("predicted value: ", prediction)

In [167]:
x=np.array([[0,0], [0,1], [1,0],[1,1]])
o_teach = np.array([0,1,1,1])
print(encodeInputs(o_teach))
print(encodeInputs(x))
weights = train(x,o_teach,20,0.00055)
# one_curr = 107
# zero_curr = 58
# out=o_teach
# out=np.multiply(out,one_curr-zero_curr)
# out=np.add(out,zero_curr)
# out
#out[1].type
# print(isinstance(out[1], np.integer))

[25. 51. 51. 51.]
[[25. 25.]
 [25. 51.]
 [51. 25.]
 [51. 51.]]
---------------training---------------
epoch:  0
[0.34375 0.34375]
[1.045  1.7743]
[2.47555 2.47555]
[3.9061 3.9061]
epoch:  1
[4.24985 4.24985]
[4.9511 5.6804]
[6.38165 6.38165]
[7.8122 7.8122]
epoch:  2
[8.15595 8.15595]
[8.8572 9.5865]
[10.28775 10.28775]
[11.7183 11.7183]
epoch:  3
[12.06205 12.06205]
[12.7633 13.4926]
[14.19385 14.19385]
[15.6244 15.6244]
epoch:  4
[15.96815 15.96815]
[16.6694 17.3987]
[18.09995 18.09995]
[19.5305 19.5305]
epoch:  5
[19.87425 19.87425]
[20.5755 21.3048]
[22.00605 22.00605]
[23.4366 23.4366]
epoch:  6
[23.78035 23.78035]
[24.4816 25.2109]
[25.91215 25.91215]
[27.3427 27.3427]
epoch:  7
[27.68645 27.68645]
[28.3877 29.117 ]
[29.81825 29.81825]
[31.2488 31.2488]
epoch:  8
[31.59255 31.59255]
[32.2938 33.0231]
[33.72435 33.72435]
[35.1549 35.1549]
epoch:  9
[35.49865 35.49865]
[36.1999 36.9292]
[37.63045 37.63045]
[39.061 39.061]
epoch:  10
[39.40475 39.40475]
[40.106  40.8353]
[41.53655 4

In [122]:
sample1 = np.array([0,0])
sample2 = np.array([1,0])
sample3 = np.array([0,1])
sample4 = np.array([1,1])
np.dot(sample2, weights)
predict(sample1, weights, 30)
predict(sample2, weights, 30)
predict(sample3, weights, 30)
predict(sample4, weights, 30)

input:  [0 0]
predicted value:  0
input:  [1 0]
predicted value:  0
input:  [0 1]
predicted value:  1
input:  [1 1]
predicted value:  1


In [60]:
from sklearn.datasets import load_digits
 
digits = load_digits()
print(digits.data.shape)
import matplotlib.pyplot as plt 

(1797, 64)


In [61]:
#flattening to 1D arrays
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))
#partitioning to training set, testing set, validation set
from sklearn.model_selection import train_test_split
X_train, X_other, y_train, y_other = train_test_split(data, digits.target, test_size=0.2, shuffle=False)
X_test, X_val, y_test, y_val = train_test_split(X_other, y_other, test_size=0.5, shuffle=False)
print("Numbers to train: " + str(len(X_train)))
print("Number of train's target: " + str(len(y_train)))
print("Numbers to test: " + str(len(X_test)))
print("Number of test's target: " + str(len(y_test)))
print("Numbers to validate: " + str(len(X_val)))
print("Number of validate's target: " + str(len(y_val)))

Numbers to train: 1437
Number of train's target: 1437
Numbers to test: 180
Number of test's target: 180
Numbers to validate: 180
Number of validate's target: 180


In [142]:
onlyones=np.zeros([len(y_train)])

for i in range(len(y_train)):
    if y_train[i]==1:
        onlyones[i]=1
newx=np.asarray(X_train, dtype=np.float32)
teachn=np.asarray(onlyones, dtype=np.float32)


In [161]:
#weights = train(newx[0:5],teachn[0:5],20,0.00055)
c=encodeInputs(newx[0])
newx[0]


array([ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.,  0.,  0., 13., 15., 10.,
       15.,  5.,  0.,  0.,  3., 15.,  2.,  0., 11.,  8.,  0.,  0.,  4.,
       12.,  0.,  0.,  8.,  8.,  0.,  0.,  5.,  8.,  0.,  0.,  9.,  8.,
        0.,  0.,  4., 11.,  0.,  1., 12.,  7.,  0.,  0.,  2., 14.,  5.,
       10., 12.,  0.,  0.,  0.,  0.,  6., 13., 10.,  0.,  0.,  0.],
      dtype=float32)

In [162]:
c=encodeInputs(newx[0])
newx[0]
c

array([25., 25., 25., 51., 51., 25., 25., 25., 25., 25., 51., 51., 51.,
       51., 25., 25., 25., 25., 51., 25., 25., 51., 51., 25., 25., 25.,
       51., 25., 25., 51., 51., 25., 25., 25., 51., 25., 25., 51., 51.,
       25., 25., 25., 51., 25., 25., 51., 25., 25., 25., 25., 51., 25.,
       51., 51., 25., 25., 25., 25., 25., 51., 51., 25., 25., 25.])

In [150]:
y_train[0]

0